In [1]:
import seaborn as sns
%matplotlib inline
import folium
import numpy as np
from matplotlib.colors import LogNorm
import pandas as pd

In [2]:
df = pd.read_csv('../data/mapping_data/avg_rainfall.csv')
df.head()

,Unnamed: 0,lat,long,avg_rainfall
0,0,14.826494,-17.375158,1.201168
1,1,15.107157,-17.124348,1.240315
2,2,14.850534,-17.084277,1.289794
3,3,14.593885,-17.044908,1.498279
4,4,14.337212,-17.006235,1.637031


In [11]:
import matplotlib.cm as cm
import matplotlib

def convert_to_hex(rgba_color):
    red = int(rgba_color[0]*255)
    green = int(rgba_color[1]*255)
    blue = int(rgba_color[2]*255)
    return '#{r:02x}{g:02x}{b:02x}'.format(r=red,g=green,b=blue)

from matplotlib.colors import LogNorm


def color(x, scalemin=df['avg_rainfall'].min(), scalemax=df['avg_rainfall'].max(), log=True):
    if log:
        norm=LogNorm(vmin=scalemin, vmax=scalemax)
    else:
        norm =matplotlib.colors.Normalize(vmin=scalemin, vmax=scalemax)
    cmap = cm.RdYlGn
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    return convert_to_hex(m.to_rgba(x))
    


In [20]:
af = folium.Map(location=[-4, 21], zoom_start=3, tiles='cartodbpositron')

plist = []
for lat, long in zip(df.lat.values,df.long.values):
    plist.append([lat,long])

#from folium.plugins import HeatMap
#HeatMap(df.values.tolist(), radius=5, blur=0, gradient={0.0: 'red', 0.65: 'lime', 1: 'blue'}).add_to(af)

downsample = 100
for idx, p in enumerate(plist[::downsample]):
    rain = df.avg_rainfall.values[idx*downsample]
    #print(idx*downsample,p, rain)
    col_idx = color(rain, log=False)
    folium.CircleMarker(location=p,radius=2, popup="%0.1f inches" %rain, fill_color=col_idx, color=col_idx).add_to(af)
 
af

In [ ]:
import datashader as ds
from datashader import transfer_functions as tf
from datashader.colors import Greys9
Greys9_r = list(reversed(Greys9))[:-2]

plot_width  = int(750)
plot_height = int(plot_width//1.2)
#x_range = (0,100)
#y_range = 


cvs = ds.Canvas(plot_width=plot_width, plot_height=plot_height)#, x_range=x_range, y_range=y_range)
agg = cvs.points(df, 'long','lat', ds.mean('avg_rainfall'))
#print(list(agg))


In [ ]:
img = tf.shade(agg, cmap=['red','yellow', 'green'], how='log')
print(type(img))
img